**Neural Network SMS Text Classifier**

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
from sklearn.pipeline import make_pipeline
import string
import re
from sklearn.model_selection import GridSearchCV


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load data into pandas DataFrame
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

In [ ]:
df = train_data.head()
df

In [ ]:
# Split the data into features (X) and labels (y)
X_train = train_data['message']
y_train = train_data['label']
X_test = test_data['message']
y_test = test_data['label']

In [ ]:
# Create a function to preprocess the text (lowercase and remove punctuation)
def preprocess_text(text):
    text = text.lower()

    text = re.sub(f"[{string.punctuation}]", "", text)

    return text

In [ ]:
# Create a pipeline that combines TfidfVectorizer and MultinomialNB
model = make_pipeline(
    TfidfVectorizer(stop_words='english', max_features=5000, ngram_range=(1, 2)),
    MultinomialNB(alpha=0.1) )

In [ ]:
# Hyperparameter tuning using GridSearchCV for the vectorizer and classifier
param_grid = {
    'tfidfvectorizer__max_features': [5000, 10000],
    'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)],
    'multinomialnb__alpha': [0.1, 1.0, 2.0]
}


In [ ]:
# Train the model using GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best model after hyperparameter tuning
best_model = grid_search.best_estimator_

In [ ]:
# Evaluate the best model
accuracy = best_model.score(X_test, y_test)
print(f"Improved Model Accuracy: {accuracy * 100:.2f}%")


In [ ]:
# Function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    pred_text = preprocess_text(pred_text)

    # Ensure the model is fitted before predicting
    prob = best_model.predict_proba([pred_text])[0]
    ham_prob = prob[0]
    spam_prob = prob[1]

    if spam_prob > ham_prob:
        return [spam_prob, "spam"]
    else:
        return [ham_prob, "ham"]

# Test the prediction function with an example
pred_text = "How are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
